In [1]:
# Langsmith with vector test
''' https://www.langchain.com/langsmith '''
''' reference : https://hellollama.net/llm-%EC%B4%88%EA%B8%89%EA%B0%95%EC%A2%8C-7-langsmith-%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0/'''

' reference : https://hellollama.net/llm-%EC%B4%88%EA%B8%89%EA%B0%95%EC%A2%8C-7-langsmith-%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0/'

In [2]:
# !pip install langchain
# !pip install huggingface_hub transformers datasets

In [3]:
#!pip install langchain-huggingface

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_teddynote import logging

logging.langsmith("RAG-INSERT_DOC")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG-INSERT_DOC


In [6]:
# import .env
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HuggingFace Access KEY'

# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)
# os.environ['HF_HOME'] = '/home/jovyan/work/tmp'
# os.environ['HF_HOME'] = 'LLM 모델을 다운로드 받을 경로'

In [7]:
''' langsmith .env/and set environment in GitHub Repository (Vector_DB_with_LLM)'''
'''
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="<your-api-key>"
LANGCHAIN_PROJECT="RAG-INSERT_DOC"
HUGGINGFACEHUB_API_TOKEN="<your-api-key>"
'''

'\nLANGCHAIN_TRACING_V2=true\nLANGCHAIN_ENDPOINT="https://api.smith.langchain.com"\nLANGCHAIN_API_KEY="<your-api-key>"\nLANGCHAIN_PROJECT="RAG-INSERT_DOC"\nHUGGINGFACEHUB_API_TOKEN="<your-api-key>"\n'

In [8]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain import LLMChain

C:\Users\euiyoung.hwang\Git_Workspace\Vector_DB_with_LLM\.venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [9]:
import sys
sys.path.append("C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Langchain/workflow/")
''' my langchain document loader package'''
from text_loader import loader_text

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
path = "C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data/"
# path = "C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Langchain/workflow/jupyter-workflow/"

In [11]:
# splits = loader_text("{}/{}".format(path, "Sample.hwp"), create_json=False)
# splits = loader_text("{}/{}".format(path, "train.pdf"), create_json=False, chunks="recursive")
splits = loader_text("{}/{}".format(path, "asiabrief_3-26.pdf"), create_json=False, chunks="recursive")
# print(f"Langchain Document : {splits}")

Loading C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf
File: C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf
split_extension: ['C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26', 'pdf']
extension: pdf
***
type : <class 'list'> / len : 5
***



type(document) <class 'list'>


In [12]:
embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
    )

In [13]:
''' print out for same sentence'''
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.049921195954084396, 0.02112860232591629, -0.05681934952735901]

In [14]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [15]:
# for doc in vectorstore.similarity_search("구글"):
for doc in vectorstore.similarity_search("저출산"):
    print(doc.page_content)

IT(Information Technology) 강국 등으로 알려져 있지만, <그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구 초저출산현상 극복과 인구구조 변화 대응 # 이삼식 한양대학교 한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지속은 인구의 지속가능성에 위협이 되고 있다. 저출산 대책은 다양 한 복합적인 원인들을 해소할 수
세계를 잇다, 미래를 빚다! 4 지대를 없애도록 한다. 무엇보다도 출산과 양육에 친화적인 직장문 화를 조성하여야 한다. 인구구조 변화에 대응해야 우리나라는 이미 20년 이상 동안 초저출산현상(합계출산율 1.3명 이하)을 겪고 있다. 적어도 당분간 ‘저출산의 덫’에서 빠져나올 수 있 을 것으로 기대하기 어렵다. 저출산 대책들이 실효성을 거두기 시작 할지라도
현상을 극복한 국가들로 알려져 있다. 우리나라는 15년 이상 저출산 대책이 추진되고 있음에도 불구 하고 출산율은 오히려 더 낮아져 최근에는 0명대를 기록하고 있다. 저출산 대책은 다양한 복합적인 원인들을 해소할 수 있도록 종합적 으로 그리고 장기간에 걸쳐 일관성 있게 추진되어야 한다. 그러나 우 리나라 저출산 대책에는 반드시 필요한 정책들이 누락되어 있는
문 제, 필요한 정책들이 포함되고 있을지라도 사각지대가 크거나 급여 등이 충분하지 않은 문제들을 가지고 있다. 이와 관련, 초저출산현상을 멈추기 위해서는 세 가지의 근본적인 노 력이 긴요하다. 첫 번째로 과도한 경쟁사회를 지양하기 위하여 노동시장을 개혁하여 야 한다. 사실 한국 사회에서 만연하는 저출산 원인들은 다소 차이가 있을지라도 OECD 국가나


In [16]:
retriever = vectorstore.as_retriever()

In [17]:
# retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")
retriever.invoke("한국의 출산율은?")

[Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 0}, page_content='Seoul National University Asia Center 1 2023년 5월 22일, 3권 26호 (통권 116호) 한국의 출산율은 어디가 바닥인가? 0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하 지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 보유하기 위해서는 합계출산율이 인구대체수준인'),
 Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 0}, page_content='IT(Information Technology) 강국 등으로 알려져 있지만, <그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구 초저출산현상 극복과 인구구조 변화 대응 # 이삼식 한양대학교 한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지속은 인구의 지속가능성에 위협이 되고 있다. 저출산 대책은 다양 한 복합적인 원인들을 해소할 수'),
 Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 1}, page_content='787/8272fb01-en (Accessed on 09 May 2023) 주) 한국은 합계출산율을 2022년까지 발표하였으나, OECD 국가 대부분 2020년까지 발표하였음. 따라서, 2020년 OECD 기준 한국의 합계출산율은 0.84임.'),
 Document(metadata={'source': 

In [18]:
from langchain.llms import HuggingFaceHub

In [19]:
# print(os.environ.get("HUGGINGFACEHUB_API_TOKEN"))

In [20]:
# llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
# model_id = 'EleutherAI/pythia-160m'
''' https://teddylee777.github.io/langchain/langchain-tutorial-02/ '''
# model_id='mistralai/Mistral-7B-v0.1'
model_id='HuggingFaceH4/zephyr-7b-beta'
llm = HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature": 0.2, "max_length": 1280})
# llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")

In [21]:
# template = """질문: {question}
# 답변: """
template = """
질문: {question}
답변: """
# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "what is elasticsearch?"
# print(llm_chain.run(question=question))
# llm_chain.run(question=question)
response = llm_chain.invoke(question)
# print(response)
# print(response['text'])
print(response['text'].split('답변:')[1].strip())

Elasticsearch is a distributed, RESTful search and analytics engine, built on top of Lucene. It provides a powerful and flexible query DSL, hit highlighting, faceted search, near real time search, rich document and field mappings, multitenancy, language-specific analyzers, and much more.

Elasticsearch is designed to be scalable horizontally, making it easy to start with a small cluster and expand as the data and search requirements grow. It is also designed to be highly available, with built-in mechanisms for data replication and recovery.

Elasticsearch is open source and released under the Apache 2.0 license. It is written in Java and runs on the JVM.

Elasticsearch is part of the Elastic Stack, which also includes Beats, Logstash, and Kibana. Together, these tools form a powerful and flexible platform for centralized logging, monitoring, and analysis.

Elasticsearch is used by a wide range of organizations, from small startups to large enterprises, in a variety of use cases, from s

In [22]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [23]:
llm = ChatHuggingFace(llm=llm)

In [24]:
# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [25]:
from langchain_teddynote.messages import stream_response

In [26]:
# question = "삼성전자가 자체 개발한 AI 의 이름은?"
question = "한국의 출산율은 얼마쯤 되나요?"
# response = chain.invoke(question)
# print(response)
response = chain.stream(question)
stream_response(response)

print('\n\n\n**')
response = llm.invoke(question)
print(response)



<|user|>
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
[Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 0}, page_content='Seoul National University Asia Center 1 2023년 5월 22일, 3권 26호 (통권 116호) 한국의 출산율은 어디가 바닥인가? 0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하 지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 보유하기 위해서는 합계출산율이 인구대체수준인'), Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 1}, page_content='787/8272fb01-en (Accessed on 09 May 2023) 주) 한국은 합계출산율을 2022년까지 발표하였으나, OECD 국가 대부분 2020년까지 발표하였음. 따라서, 2020년 OECD 기준 한국의 합계출산율은 0.84임.'), Document(metadata={'source': 'C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data//asiabrief_3-26.pdf', 'page': 0}, page_content='IT(Informat

In [27]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)
question = "한국의 출산율은?"
result = qa.invoke({"query": question})
print(result)
print(result['result'])


{'query': '한국의 출산율은?', 'result': "<|user|>\nYou are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nAnswer in Korean.\n\n#Context: \nSeoul National University Asia Center 1 2023년 5월 22일, 3권 26호 (통권 116호) 한국의 출산율은 어디가 바닥인가? 0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하 지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 보유하기 위해서는 합계출산율이 인구대체수준인\n\nIT(Information Technology) 강국 등으로 알려져 있지만, <그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구 초저출산현상 극복과 인구구조 변화 대응 # 이삼식 한양대학교 한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지속은 인구의 지속가능성에 위협이 되고 있다. 저출산 대책은 다양 한 복합적인 원인들을 해소할 수\n\n787/8272fb01-en (Accessed on 09 May 2023) 주) 한국은 합계출산율을 2022년까지 발표하였으나, OECD 국가 대부분 2020년까지 발표하였음. 따라서, 2020년 OECD 기준 한국의 합계출산율은 0.84임.\n\n출산율 회복과 인구구조 변화 대응은 양자택일이 아닌 양자병행 과제 최근 한국 사회에서 출산율이 0명대로 급락하면서 출산율 회복에 대 한 회의감이 증가하고 있다. 그 영향으로 출산율 회복을 위한 의지와 노력이 약화되고 대신 인구감소 및 고령화에 적응하여야 한다는 논 의가 활발하다. 그러나 출산율 회복 없는 인구변화에 대한 대응은 높 은 비용과 